In [2]:
import tweepy
from twitter_authentication import bearer_token
import time
import pandas as pd
import json 
import pickle
import folium
from folium.plugins import MarkerCluster
import json
import csv
from pathlib import Path
import glob, os, json


In [63]:
def json_load(filepath):
    with open(filepath, "r") as json_file:
        data = json.load(json_file)
    return data

def json_dump(data, filepath, pretty_format = True):
    with open(filepath, 'w') as fw:
        if pretty_format:
            json.dump(data, fw, indent=2, sort_keys=True)
        else:
            json.dump(data, fw)

def pickle_dump(obj, pickle_filepath):
    with open(pickle_filepath, "wb") as f:
        pickle.dump(obj, f, protocol=2)

def pickle_load(pickle_filepath):
    with open(pickle_filepath, "rb") as f:
        obj = pickle.load(f)
    return obj

In [3]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [74]:
def get_tweets(place_dict = {},
               user_dict = {},
               tweet_dict = {},
                next_token = None,
               alz_tweets = [], 
                query = 'alzheimers disease -is:retweet place_country:US',
              user_fields = ['username', 'public_metrics', 'description', 'location'],
              place_fields = ['place_type', 'geo'],
              tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
              expansions = ['author_id', 'geo.place_id'],
              start_time = '2010-01-01T00:00:00Z',
               end_time = '2023-01-15T00:00:00Z'
              ):
    place_keys= ["full_name", 'id', 'contained_within', 'country', 'country_code', 'geo', 'name', 'place_type' ]
    user_keys= ['id', 'name', 'username', 'created_at', 
            'description', "entities", 'location', 'pinned_tweet_id', 
            'profile_image_url', 'protected', 'public_metrics', 'url', 'verified', 'withheld']
    tweet_keys = ['id', 'text', 'author_id', 'context_annotations',
             'conversation_id', 'entities', 'in_reply_to_user_id', 'lang',
             'non_public_metrics', 'organic_metrics', 'possibly_sensitive',
             'promoted_metrics', 'public_metrics', 'referenced_tweets', 'reply_settings',
             'source', 'withheld']    


    for tweet in tweepy.Paginator(client.search_all_tweets, 
                                     query = query,
                                     user_fields = user_fields,
                                     place_fields = place_fields,
                                     tweet_fields = tweet_fields,
                                     expansions = expansions,
                                     start_time = start_time,
                                     end_time = end_time,
                                  pagination_token = next_token,
                                  max_results=500):
        time.sleep(1)
        alz_tweets.append(tweet)
    
    
    # Loop through each response object
    for response in alz_tweets:
        # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
        for place in response.includes['places']:
            place_obj = {}
            for key in place_keys:
                place_obj[key] = place[key]
            place_dict[place_obj['id']] = place_obj
        for user in response.includes['users']:
            user_obj = {}
            for key in user_keys:
                user_obj[key] = user[key]
            user_dict[user_obj['id']] = user_obj

        for tweet in response.data:
            tweet_obj = {}
            for key in tweet_keys:
                tweet_obj[key] = getattr(tweet, key)
            tweet_dict[tweet_obj['id']] = tweet_obj
    
    return place_dict, user_dict, tweet_dict, next_token, alz_tweets



In [75]:
next_token = None
place_dict = {}
user_dict = {}
tweet_dict = {}
alz_tweets = []



place_dict, user_dict, tweet_dict, next_token, alz_tweets = get_tweets(place_dict,user_dict,tweet_dict, next_token, alz_tweets)

In [76]:
response = alz_tweets[0]

In [52]:
# print(len(response.data))
# for tweet in response.data:
#     print(dict(tweet))

342
{'created_at': datetime.datetime(2023, 1, 13, 15, 50, 35, tzinfo=datetime.timezone.utc), 'text': 'Neuroscientists reveal how a toxic form of tau protein, #taupT217, spreads through the brain as #Alzheimers disease progresses\n\nhttps://t.co/LkvPgdlntB', 'geo': {'place_id': 'ab2f2fac83aa388d'}, 'author_id': 1036037388, 'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0, 'impression_count': 44}, 'edit_history_tweet_ids': [1613926491200552961], 'id': 1613926491200552961}
{'created_at': datetime.datetime(2023, 1, 12, 14, 49, 54, tzinfo=datetime.timezone.utc), 'text': "New Alzheimer's medication could add 'truly valuable' time to patient lives \n@7NewsDC @alzassociation @alzheimerssoc \nhttps://t.co/uHJMSVXSLN", 'geo': {'place_id': '01fbe706f872cb32'}, 'author_id': 19302873, 'public_metrics': {'retweet_count': 1, 'reply_count': 0, 'like_count': 1, 'quote_count': 0, 'impression_count': 1041}, 'edit_history_tweet_ids': [1613548831991386117], 'id': 1

In [77]:
tweets_data = []
for tweet in response.data:
    #print(dict(tweet))
    for place in response.includes['places']:
        if tweet.geo is not None:
            if tweet.geo['place_id'] == place['id']:
                tweet_info = {
                    "content": tweet.text,
                    "geo_id": place['id'] if place is not None else None,
                    "geo_info": dict(place)
                }
                tweets_data.append(tweet_info)
                #print(dict(place))

In [78]:
print(len(tweets_data))

303


In [110]:
# for tweet in tweets_data:
#     print(tweet)
#     print()

In [79]:
json_dump(tweets_data, filepath = "Full_List/alzheimers_disease.json")

In [86]:
def get_tweets2(place_dict = {},
               user_dict = {},
               tweet_dict = {},
                next_token = None,
               alz_tweets = [],
                query = 'alzheimer\'s disease -is:retweet place_country:US',
              user_fields = ['username', 'public_metrics', 'description', 'location'],
              place_fields = ['place_type', 'geo'],
              tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
              expansions = ['author_id', 'geo.place_id'],
              start_time = '2010-01-01T00:00:00Z',
               end_time = '2023-01-15T00:00:00Z'
              ):
    place_keys= ["full_name", 'id', 'contained_within', 'country', 'country_code', 'geo', 'name', 'place_type' ]
    user_keys= ['id', 'name', 'username', 'created_at', 
            'description', "entities", 'location', 'pinned_tweet_id', 
            'profile_image_url', 'protected', 'public_metrics', 'url', 'verified', 'withheld']
    tweet_keys = ['id', 'text', 'author_id', 'context_annotations',
             'conversation_id', 'entities', 'in_reply_to_user_id', 'lang',
             'non_public_metrics', 'organic_metrics', 'possibly_sensitive',
             'promoted_metrics', 'public_metrics', 'referenced_tweets', 'reply_settings',
             'source', 'withheld']    


    for tweet in tweepy.Paginator(client.search_all_tweets, 
                                     query = query,
                                     user_fields = user_fields,
                                     place_fields = place_fields,
                                     tweet_fields = tweet_fields,
                                     expansions = expansions,
                                     start_time = start_time,
                                     end_time = end_time,
                                  pagination_token = next_token,
                                  max_results=500):
        time.sleep(1)
        alz_tweets.append(tweet)
    
    
    return alz_tweets



In [88]:
next_token = None
place_dict = {}
user_dict = {}
tweet_dict = {}
alz_tweets = []



alz_tweets = get_tweets2(place_dict,user_dict,tweet_dict, next_token, alz_tweets)

In [89]:
response = alz_tweets[0]

In [91]:
tweets_data = []
for tweet in response.data:
    #print(dict(tweet))
    for place in response.includes['places']:
        if tweet.geo is not None:
            if tweet.geo['place_id'] == place['id']:
                tweet_info = {
                    "content": tweet.text,
                    "geo_id": place['id'] if place is not None else None,
                    "geo_info": dict(place)
                }
                tweets_data.append(tweet_info)
                #print(dict(place))

In [96]:
json_dump(tweets_data, filepath = "Full_List/alzheimer\'s_disease.json")

In [108]:
def get_tweets3(place_dict = {},
               user_dict = {},
               tweet_dict = {},
                next_token = None,
               alz_tweets = [],
                query = 'alzheimer -is:retweet place_country:US',
              user_fields = ['username', 'public_metrics', 'description', 'location'],
              place_fields = ['place_type', 'geo'],
              tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
              expansions = ['author_id', 'geo.place_id'],
              start_time = '2010-01-01T00:00:00Z',
               end_time = '2023-01-15T00:00:00Z'
              ):
    place_keys= ["full_name", 'id', 'contained_within', 'country', 'country_code', 'geo', 'name', 'place_type' ]
    user_keys= ['id', 'name', 'username', 'created_at', 
            'description', "entities", 'location', 'pinned_tweet_id', 
            'profile_image_url', 'protected', 'public_metrics', 'url', 'verified', 'withheld']
    tweet_keys = ['id', 'text', 'author_id', 'context_annotations',
             'conversation_id', 'entities', 'in_reply_to_user_id', 'lang',
             'non_public_metrics', 'organic_metrics', 'possibly_sensitive',
             'promoted_metrics', 'public_metrics', 'referenced_tweets', 'reply_settings',
             'source', 'withheld']    


    for tweet in tweepy.Paginator(client.search_all_tweets, 
                                     query = query,
                                     user_fields = user_fields,
                                     place_fields = place_fields,
                                     tweet_fields = tweet_fields,
                                     expansions = expansions,
                                     start_time = start_time,
                                     end_time = end_time,
                                  pagination_token = next_token,
                                  max_results=500):
        time.sleep(1)
        alz_tweets.append(tweet)
    
    
    return alz_tweets



In [109]:
next_token = None
place_dict = {}
user_dict = {}
tweet_dict = {}
alz_tweets = []



alz_tweets = get_tweets3(place_dict,user_dict,tweet_dict, next_token, alz_tweets)

Rate limit exceeded. Sleeping for 75 seconds.


In [99]:
response = alz_tweets[0]

In [100]:
tweets_data = []
for tweet in response.data:
    #print(dict(tweet))
    for place in response.includes['places']:
        if tweet.geo is not None:
            if tweet.geo['place_id'] == place['id']:
                tweet_info = {
                    "content": tweet.text,
                    "geo_id": place['id'] if place is not None else None,
                    "geo_info": dict(place)
                }
                tweets_data.append(tweet_info)
                #print(dict(place))

In [102]:
json_dump(tweets_data, filepath = "Full_List/alzheimer.json")

In [103]:
def get_tweets3(place_dict = {},
               user_dict = {},
               tweet_dict = {},
                next_token = None,
               alz_tweets = [],
                query = 'alzheimers -is:retweet place_country:US',
              user_fields = ['username', 'public_metrics', 'description', 'location'],
              place_fields = ['place_type', 'geo'],
              tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
              expansions = ['author_id', 'geo.place_id'],
              start_time = '2010-01-01T00:00:00Z',
               end_time = '2023-01-15T00:00:00Z'
              ):
    place_keys= ["full_name", 'id', 'contained_within', 'country', 'country_code', 'geo', 'name', 'place_type' ]
    user_keys= ['id', 'name', 'username', 'created_at', 
            'description', "entities", 'location', 'pinned_tweet_id', 
            'profile_image_url', 'protected', 'public_metrics', 'url', 'verified', 'withheld']
    tweet_keys = ['id', 'text', 'author_id', 'context_annotations',
             'conversation_id', 'entities', 'in_reply_to_user_id', 'lang',
             'non_public_metrics', 'organic_metrics', 'possibly_sensitive',
             'promoted_metrics', 'public_metrics', 'referenced_tweets', 'reply_settings',
             'source', 'withheld']    


    for tweet in tweepy.Paginator(client.search_all_tweets, 
                                     query = query,
                                     user_fields = user_fields,
                                     place_fields = place_fields,
                                     tweet_fields = tweet_fields,
                                     expansions = expansions,
                                     start_time = start_time,
                                     end_time = end_time,
                                  pagination_token = next_token,
                                  max_results=500):
        time.sleep(1)
        alz_tweets.append(tweet)
    
    
    return alz_tweets



In [104]:
next_token = None
place_dict = {}
user_dict = {}
tweet_dict = {}
alz_tweets = []



alz_tweets = get_tweets2(place_dict,user_dict,tweet_dict, next_token, alz_tweets)

In [105]:
response = alz_tweets[0]

In [106]:
tweets_data = []
for tweet in response.data:
    #print(dict(tweet))
    for place in response.includes['places']:
        if tweet.geo is not None:
            if tweet.geo['place_id'] == place['id']:
                tweet_info = {
                    "content": tweet.text,
                    "geo_id": place['id'] if place is not None else None,
                    "geo_info": dict(place)
                }
                tweets_data.append(tweet_info)
                #print(dict(place))

In [107]:
json_dump(tweets_data, filepath = "Full_List/alzheimers.json")